In [1]:
import csv
import numpy as np
import pandas as pd
import random
import time
import datetime
from tqdm.notebook import *

import bipartite_algo
from request import *
from vehicle import *
from Waypoint import WayPoint, MapSystem


for day in trange(10, 32):
    '''Parameter Setting'''
    eps = 1e-6
    cardinal_const = 1000000

    '''Simulation Setup'''
    period_length = 30 #i.e. assignment is done each 30-secs
    H = 20 #i.e. number of periods ############### MOVING to MUlTIPERIOD
    CAPACITY = 4 #i.e. maximum capacity of vehicles
    MAX_WAITING_TIME = 240 #i.e. maximum waiting time for request before picked up
    c = 5 #i.e. weightage for trip benefits (active time) compared to the loss (cruising time)
    base_utility = 0 #1e8

    REMOVAL = 500 #i.e. set the allowance for trip removal to manage the network sparsity problems
    ratio = 1.2
    divisor = 1 #i.e. ratio, divisor to set the balance between HIGH and LOW activity vehicles
    seed = 0
    #day = 7
    HOUR = 17
    filter_treshold = 10 #to gauge the assignment potential based on reachability of vehicle initial locations (nodes)
    w = 0 #{0 - efficient allocation, 1 - fair allocation}

    DATA_TYPE = 'WHOLE' #i.e. {'WHOLE': whole Manhattan area, 'REGION': parts of the map}
    if DATA_TYPE == 'WHOLE':
        MIN_LIST = [-float("inf")]
        MAX_LIST = [float("inf")]
    else:
        MIN_LIST = [-float("inf")] + list(np.linspace(40.72, 40.74, 3)) + [40.75]
        MAX_LIST = [40.75] + list(np.linspace(40.72, 40.74, 3) + .035) + [float("inf")]


    #------------------------------------------------------------------------------
    '''Generate the road network'''

    manhat_point = pd.read_pickle('..\\..\\Data\\RoadNetwork\\manhat_point.pkl')
    manhat_edge = pd.read_pickle('..\\..\\Data\\RoadNetwork\\manhat_edge.pkl')

    TMP_msg = manhat_edge.values
    NodePos = {}
    NearbyNode = {} #Neighbors

    for rd_msg in TMP_msg:
        p = int(rd_msg[0])
        q = int(rd_msg[1])
        NodePos[p] = rd_msg[2]
        NodePos[q] = rd_msg[3]

        if NearbyNode.get(p) == None:
            NearbyNode[p] = [q]
        else:
            NearbyNode[p] += [q]

    mapsystem = MapSystem()
    mapsystem._node_dict = NodePos
    mapsystem._nearby_node = NearbyNode

    #------------------------------------------------------------------------------
    '''Generate the inter-node travel time'''

    tt_dict = {}
    node_dict = {}
    tt_list = []

    traveltime = pd.read_csv('..\\..\\Data\\TravelTimePrediction\\time_cost' + str(HOUR) + '.csv', index_col=[0, 1], header=None)
    traveltime.columns = [HOUR]
    tmp_list = list(traveltime[HOUR])
    tot_num = 0

    for n1, n2 in zip(traveltime.index.get_level_values(0), traveltime.index.get_level_values(1)):

        tt_dict[(n1, n2)] = tmp_list[tot_num]
        node_dict[(n1,n2)] = tot_num
        tot_num += 1        

    mapsystem.update_distance(cur_time = 0, distance = tt_dict) #i.e. distance = travel time (in secs)

    tt_list += [list(tt_dict.values())]

    #------------------------------------------------------------------------------
    '''Reading Request Data'''

    REQ_DATA = pd.read_pickle('..\..\\Data\\RequestGenerator\\May_2013_HOUR17_singlebatch.pkl')
    #------------------------------------------------------------------------------
    '''Function lists:
        1. closest_node - convert geocoordinates to nearest point index in the road network
        2. status_update - update vehicles and requests' attributes & some arrays 
        for route recording (see l.116-124, 133-140)
        3. compute_routing - take in available taxis and coming requests & output 
        optimal (dependent on specified efficiency-fairness conditions) routes'''

    def closest_node(loc):

        global manhat_point

        loc = np.array([loc])
        nodes = np.array(list(manhat_point['Coordinate']))
        dist_2 = np.sum((nodes - loc)**2, axis=1)
        pos = np.argmin(dist_2)

        return manhat_point.index[pos]

    def status_update(t, V_STORAGE, FIN_ALLOC, STACKED_ROUTE, STACKED_TIME):

        for vid in FIN_ALLOC:

            cur_v = V_STORAGE[vid]
            destination_list = FIN_ALLOC[vid] #i.e. the assigned routing for vehicle with ID vid
            moving_time = t - cur_v._overall_time #i.e. variable for 'remaining time'; starting with 30 secs (period length)
            prev_moving_time = moving_time

            for (rid, p_or_d) in destination_list: 
                #i.e. rid: request ID, p_or_d: 0-pickup/1-dropoff

                cur_r = REQ_STORAGE[rid]

                if moving_time < 0:
                    break

                if p_or_d == 0:

                    reached, moving_time = mapsystem.move_to_destination(cur_v._waypoint, cur_r._origin_waypoint, moving_time)
                    #i.e. reached: binary variable; indication if vehicle cur_v has reached the specified node in the destination_list,
                    #moving_time: reduced by the traveltime from cur_v's 'current' position (cur_v._waypoint)
                    #to the specified PICKUP node (cur_r._origin_waypoint) in destination_list

                    if len(cur_v._boarded_requests) != 0:
                        cur_v._active_timecost += prev_moving_time - moving_time
                        cur_r.update_trip_value(0, moving_time - prev_moving_time)

                    if reached:
                        cur_v.picking_up(cur_r)                    
                        cur_r.picking_up(t - moving_time)
                        STACKED_ROUTE[vid] += [(rid, p_or_d)]
                        STACKED_TIME[vid] += [t - moving_time]

                    else:
                        break

                if p_or_d == 1:
                    reached, moving_time = mapsystem.move_to_destination(cur_v._waypoint,cur_r._destination_waypoint, moving_time)
                    #moving_time: see l.113-114, with destination the specified DROPOFF node (cur_r._destination_waypoint)

                    cur_v._active_timecost += prev_moving_time - moving_time
                    cur_r.update_trip_value(0, moving_time - prev_moving_time)

                    if reached:
                        cur_v.finishing(cur_r)
                        cur_r.finishing(t - moving_time)
                        STACKED_ROUTE[vid] += [(rid, p_or_d)]
                        STACKED_TIME[vid] += [t - moving_time]

                    else:
                        break

                prev_moving_time = moving_time

    def compute_routing(h, t, act_corr, V_STORAGE, REQ_STORAGE, DEMAND_LIST, FIN_ALLOC, switch = 0):

        global req_pos, \
            All_edges, Value_weight, Distances

        print('req_pos bf:', req_pos)

        for i in range(req_pos, len(REQ_STORAGE)):
            r = REQ_STORAGE[i]

            if r._request_time >= h*period_length:
                break

            DEMAND_LIST += [r]
            req_pos += 1

        print('req_pos aft:', req_pos)    

        TAXI_LIST = [v for v in V_STORAGE if v._arr_time < t]

        '''Create bipartite graph and other information to be passed into the main algorithm'''
        RTV_graph = {} #i.e. built based on constraints specified below (see l.211-212, 218-219)
        All_edges = []
        Sorting_weight = [] #i.e. for fairness; needs to account for historical values accumulated by each vehicle
        Value_weight = [] #i.e. for efficiency; needs to capture the maximization of service rate (see l.198 adding in large constant), 
        Distances = [] #
        #wlog it disregards history

        NEW_TAXI_LIST = []
        NEW_DEMAND_LIST = []
        index = -1

        for v in TAXI_LIST:
            index += 1

            '''There are boarded requests, direct addition to FIN_ALLOC'''
            if len(v._boarded_requests) > 0:
                r = v._boarded_requests[0]
                RTV_graph[v._vid] = {r._rid: [r._trip_value, r._trip_active, 0]}            
                FIN_ALLOC[v._vid] = ((r._rid, 1),)
                continue

            '''Once assigned, it commits to the assignment, direct addition to FIN_ALLOC'''
            if len(v._assigned_requests) > 0:            
                r = v._assigned_requests[0]
                RTV_graph[v._vid] = {r._rid: [r._trip_value, r._trip_active, 0]}
                FIN_ALLOC[v._vid] = ((r._rid, 0), (r._rid, 1))
                continue

            NEW_TAXI_LIST += [v]

            '''Adding in empty trip''' # no longer needed
            RTV_graph[v._vid] = {-v._vid - 1: [0, 0, 0]}
            All_edges += [[v._vid, -v._vid - 1]]

            Sorting_weight += [v._allocated_value1 + 0]
            Value_weight += [0  + base_utility]
            Distances += [base_utility]

            for r in DEMAND_LIST:

                if r._assigned == True or r._size > CAPACITY:
                    continue

                NEW_DEMAND_LIST += [r._rid]

                pickup_duration = mapsystem.distance(v._waypoint, r._origin_waypoint)
                waiting_time = (t - r._request_time) + pickup_duration

                if r.latest_acceptable_pickup_time <= t + pickup_duration:
                    continue

                else:
                    trip_active = r._trip_length
                    trip_value = trip_active - waiting_time

                    if trip_value < 0 :
                        continue

                    RTV_graph[v._vid][r._rid] = [trip_value, trip_active, pickup_duration]
                    All_edges += [[v._vid, r._rid]]

                    Sorting_weight += [v._allocated_value1 + c*trip_active - pickup_duration]
                    #Value_weight += [c*trip_active + base_utility - pickup_duration]
                    Value_weight += [c*r._trip_length + base_utility - pickup_duration]
                    Distances += [r._trip_length + pickup_duration]

        print('NEW_DEMAND_LIST, len:', len(set(NEW_DEMAND_LIST)))

        '''Recording purposes'''
        hist_act = [v._active_timecost for v in TAXI_LIST]
        hist_val = [v._allocated_value1 for v in TAXI_LIST]
        min_vid = hist_val.index(min(hist_val))

        print('Active time minmax:', [min(hist_act), max(hist_act)])
        print('activity value minmax:', [min(hist_val), max(hist_val)])
        print('vid min?', min_vid)

        global nVeh, nReq
        nVeh, nReq = len(NEW_TAXI_LIST), len(DEMAND_LIST)
        print('NUM OF AVAILABLE TAXIS:', len(NEW_TAXI_LIST))
        print('NUM OF REQUESTS:', len(DEMAND_LIST))

        '''Solving for optimal route'''
        print('testing', np.array(All_edges), np.array(Value_weight))
        """
        if len(NEW_TAXI_LIST) > 0:
            route_dict = bipartite_algo.solve_rtv_graph(h, H, RTV_graph, NEW_TAXI_LIST, DEMAND_LIST, All_edges, Sorting_weight, Value_weight, act_corr, c)
        else:
            route_dict = {}

        act_list = [v._allocated_value1 for v in V_STORAGE]
        min_driver = min(act_list)
        ilp_discr = np.std(act_list)
        value_sum = sum(act_list)

        unassigned_supply = len([vid for vid in route_dict if list(route_dict[vid]) == []])

        idle_taxis = []
        unassigned_supply = []

        '''Storing the optimal routes (i.e. route_dict) into FIN_ALLOC & updating of vehicle/request attributes'''
        for vid in route_dict:
            if list(route_dict[vid]) == []:
                unassigned_supply += [vid]

            FIN_ALLOC[vid] = route_dict[vid]
            cur_v = V_STORAGE[vid]
            cur_v.new_assigning()

            for (rid, p_or_d) in FIN_ALLOC[vid]:
                cur_r = REQ_STORAGE[rid]
                if p_or_d == 0:
                    cur_r.assigning(vid,t)
                    cur_v.assigning(cur_r)

            if cur_v.isidle:
                idle_taxis += [cur_v]

        for vid in FIN_ALLOC:
            V_STORAGE[vid]._overall_time = t

        print('------------------------------------------------------------------')
        print('UNASSIGNED TAXIS',len(unassigned_supply))
        print('IDLE TAXIS', len(idle_taxis))

        return value_sum, ilp_discr, min_driver
        """


    #------------------------------------------------------------------------------
    '''STARTING SIMULATION'''

    print('DAY:', day)
    print('TRESH:', filter_treshold)
    lat_min, lat_max = list(zip(MIN_LIST, MAX_LIST))[0]

    '''Filter the request data (first 30s, within capacity, trip length >= 400 secs)'''
    valid_data = REQ_DATA[REQ_DATA['bydates'] == datetime.date(2013,5,day)]

    ORI_REQ_NUM = len(valid_data)
    print('NUMREQ from original data:', ORI_REQ_NUM)

    chosen_tripid = [valid_data.index[i] for i in range(len(valid_data.index))]

    '''Generate artificial vehicle locations'''
    chosen_nodes = []
    for tid in chosen_tripid:
        chosen_nodes += [valid_data.loc[tid, 'pickup_nodes']]

    print('ORIGINAL TRIP NODES:', chosen_nodes)
    print('how large the duplicates? (len vs lenset):', (len(chosen_nodes), len(set(chosen_nodes))))

    rand_num = np.random.uniform(0,1,len(chosen_nodes))
    v_location_ACT = [] #i.e. to initialize locations of HIGH activity vehicles - extremize the environment: exactly on the node of chosen requests
    for i in range(len(rand_num)):
        if rand_num[i] <= divisor:
            v_location_ACT += [manhat_point.loc[chosen_nodes[i], 'Coordinate']]

    TAXI_TOTAL_NUM = int(ratio/divisor*len(v_location_ACT)) #i.e. set the ratio of high vs low activity vehicles

    node_in_areas = {} #i.e. nodes that are within 210s travelling time
    for n1 in manhat_point.index:
        node_in_areas[n1] = manhat_point.loc[n1, 'ExtendedNeighbors_210s_1']

    v_init_nodes = []
    for node in chosen_nodes:
        v_init_nodes += node_in_areas[node]

    filter_init = [] #i.e. store only the nodes with 'sufficient' reachability potential
    for vloc in set(v_init_nodes):
        potential = sum([(vloc in node_in_areas[n1]) for n1 in chosen_nodes])

        if potential <= filter_treshold:
            continue

        filter_init += [vloc]

    print('len v_init:', len(set(v_init_nodes)))
    print('HOW LARGE?', len(filter_init))
    print('REQUIRED?', TAXI_TOTAL_NUM - len(v_location_ACT))

    random.seed(seed)
    if TAXI_TOTAL_NUM - len(v_location_ACT) > len(filter_init):
        print('failed...')
        continue
    loc_id = np.random.choice(filter_init, TAXI_TOTAL_NUM - len(v_location_ACT), replace = False)
    v_location = [manhat_point.loc[filter_init[i], 'Coordinate'] for i in range(len(loc_id))] #i.e. to initialize locations for LOW activity vehicles

    '''Reset V_STORAGE'''
    V_STORAGE = np.array([])

    for loc in v_location_ACT:
        vid = len(V_STORAGE)
        newv = Vehicle(vid, loc, CAPACITY)

        newv._point_id = closest_node(newv._cur_position)
        newv._waypoint = WayPoint(newv._point_id, newv._point_id, 0, 0)
        newv._arr_time = 0

        random.seed(vid + 1 + seed)
        newv._allocated_value1 = random.uniform(200, 400) #i.e. set the HIGH historical active time

        V_STORAGE = np.append(V_STORAGE, newv)

    for loc in v_location:
        vid = len(V_STORAGE)
        newv = Vehicle(vid, loc, CAPACITY)
        newv._point_id = closest_node(newv._cur_position)
        newv._waypoint = WayPoint(newv._point_id, newv._point_id, 0, 0)

        newv._arr_time = 0

        random.seed(vid + 1 + seed)
        newv._allocated_value1 = random.uniform(50, 100) #i.e. set the LOW historical active time

        V_STORAGE = np.append(V_STORAGE, newv)             

    print('NUM OF TAXIS:', len(V_STORAGE))

    '''Reset REQ_STORAGE'''
    REQ_STORAGE=np.array([])

    for i in chosen_tripid:
        region = 'None'
        origin = valid_data.loc[i, 'pickup_coordinates']
        destination = valid_data.loc[i, 'dropoff_coordinates']
        trip_length = valid_data.loc[i, 'trip_length']
        size = valid_data.loc[i, ' passenger_count']
        arr_time = 0
        req_ID = len(REQ_STORAGE)

        newreq = Request(req_ID, region, origin, destination, arr_time, size, trip_length)

        p = valid_data.loc[i, 'pickup_nodes']
        q = valid_data.loc[i, 'dropoff_nodes']
        newreq._origin_closest_node = p
        newreq._destination_closest_node = q
        newreq._origin_waypoint = WayPoint(p, p, 0., 0.)
        newreq._destination_waypoint = WayPoint(q, q, 0., 0.)
        newreq._trip_length = mapsystem.distance(newreq._origin_waypoint, newreq._destination_waypoint)
        newreq._max_waiting_time = MAX_WAITING_TIME

        REQ_STORAGE = np.append(REQ_STORAGE, newreq)

    REQ_NUM = len(REQ_STORAGE)
    print('NUM OF REQS:', REQ_NUM)
    removal_rate = REQ_NUM/ORI_REQ_NUM

    '''Starting Simulation'''
    DEMAND_LIST = []
    STACKED_ROUTE = [[] for i in range(TAXI_TOTAL_NUM)]
    STACKED_TIME = [[] for i in range(TAXI_TOTAL_NUM)]

    req_pos = 0          
    weight_trial = [w for i in range(H)]

    print('WEIGHT:', w)
    weight_type = w

    switch = 0
    for h in range(1, H+1):

        print('PERIOD', h)
        print('===========================================')

        act_corr = weight_trial[h-1]
        t = h*period_length
        FIN_ALLOC = {}

        DEMAND_LIST = [r for r in DEMAND_LIST if r._assigned == True]

        #ILP_VALUE, ILP_DISCR, MIN_DRIVER = \
        compute_routing(h, t, act_corr, V_STORAGE, REQ_STORAGE, DEMAND_LIST, FIN_ALLOC, switch)

        list_normal = [v._allocated_value1 for v in V_STORAGE]

        if np.min(list_normal) > 0:
            switch = 1

        """
        print('normal')
        print('std:', np.std(list_normal))
        print('minmax:', (np.min(list_normal), np.max(list_normal)))
        print('----')

        if h < H:
            t = (h+1)*period_length
            status_update(t, V_STORAGE, FIN_ALLOC, STACKED_ROUTE, STACKED_TIME)

    TOT_SERVED, SERVICE_RATE, MEAN_WAIT, MEAN_DELAY, MEAN_INTRIP = [0 for i in range(5)]

    f = open('Tradeoff_w_Horizons.csv', 'a', newline = '')
    to_append = [[DATA_TYPE, REMOVAL, removal_rate, MAX_WAITING_TIME, lat_min, HOUR, H, TAXI_TOTAL_NUM, REQ_NUM, weight_trial, 
                  TOT_SERVED, SERVICE_RATE, MEAN_WAIT, MEAN_DELAY, MEAN_INTRIP,
                  np.sum(list_normal), np.std(list_normal), np.min(list_normal), np.max(list_normal)]]
    writer = csv.writer(f)
    writer.writerows(to_append)
    f.close()

    '''Record list_normal as column'''
    f = open('ActiveTime.csv', 'a', newline = '')
    to_append = [[HOUR, weight_type] + list_normal]
    writer = csv.writer(f)
    writer.writerows(to_append)
    f.close()"""

    import pickle
    #nVeh, nReq = 129, 108
    mileages = np.random.normal(2, 0.4, nVeh)  # hardcoded
    mileages = 10 ** mileages

    carbon = (mileages[np.array(All_edges)[:, 0]] * np.array(Distances)).tolist()
    eff_matrix, carb_matrix = np.zeros([nReq, nVeh+nReq])-1e6, np.zeros([nReq, nVeh+nReq])+1e6 # flip to requests to vehicles

    All_edges, Value_weight, carbon = np.array(All_edges), np.array(Value_weight), np.array(carbon)
    mask = np.where(All_edges[:, 1] < 0)  # for deletion
    print(mask)
    All_edges = np.delete(All_edges, mask, axis=0)
    Value_weight = np.delete(Value_weight, mask)
    carbon = np.delete(carbon, mask)
    print(np.where(All_edges<0))
    
    eff_matrix[All_edges[:, 1], All_edges[:, 0]] = Value_weight
    carb_matrix[All_edges[:, 1], All_edges[:, 0]] = carbon
    
    eff_matrix[np.arange(nReq), nVeh+np.arange(nReq)] = 0
    carb_matrix[np.arange(nReq), nVeh+np.arange(nReq)] = 0
    print('\n=============\n', eff_matrix, '\n\n', carb_matrix,'\n=============\n')
     
    with open(f'C:\\Users\\user\\Google Drive\\Drive\\Work\\RProj\\multiperiod_samples\\sample{day}.pickle', 'wb') as file:
        pickle.dump({
            'eff_matrix': eff_matrix,
            'carb_matrix': carb_matrix, 
        }, file)

C:\Users\user\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


DAY: 10
TRESH: 10
NUMREQ from original data: 80
ORIGINAL TRIP NODES: [762, 7525, 478, 2559, 2826, 1769, 3400, 2530, 7525, 7525, 4939, 6601, 1876, 7525, 1267, 7525, 2524, 3722, 5330, 5810, 955, 7611, 478, 7700, 5823, 8220, 2316, 5603, 4931, 7525, 4186, 2251, 7525, 7525, 4994, 478, 977, 3149, 1266, 7525, 2605, 3804, 3180, 2231, 8044, 2732, 8340, 8311, 6672, 7525, 7822, 1934, 1774, 8959, 2833, 2236, 1903, 7701, 8624, 4941, 2256, 1197, 5758, 1698, 2810, 6105, 6601, 5330, 478, 3413, 7525, 6497, 3521, 1304, 5185, 7525, 2542, 673, 2557, 7633]
how large the duplicates? (len vs lenset): (80, 64)
len v_init: 2263
HOW LARGE? 7
REQUIRED? 16
failed...


C:\Users\user\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


DAY: 11
TRESH: 10
NUMREQ from original data: 86
ORIGINAL TRIP NODES: [5445, 5222, 1330, 1481, 8302, 2557, 3809, 2452, 5559, 478, 478, 7805, 2375, 1146, 8044, 708, 1191, 5930, 2829, 127, 3883, 2161, 9689, 3564, 3195, 2829, 7653, 5741, 3205, 2521, 7645, 2717, 1808, 1933, 7762, 2161, 2256, 6263, 4544, 2819, 2529, 1304, 3688, 8018, 1016, 6263, 2529, 2904, 1230, 2165, 5928, 1390, 4257, 6745, 4653, 5811, 3162, 2825, 9616, 3480, 2244, 1982, 2653, 478, 2421, 1268, 3536, 478, 2732, 3480, 2144, 5816, 5962, 1920, 2717, 478, 3532, 5467, 4015, 4251, 6774, 478, 8311, 784, 2576, 987]
how large the duplicates? (len vs lenset): (86, 75)
len v_init: 3178
HOW LARGE? 137
REQUIRED? 17
NUM OF TAXIS: 103
NUM OF REQS: 86
WEIGHT: 0
PERIOD 1
req_pos bf: 0
req_pos aft: 86
NEW_DEMAND_LIST, len: 86
Active time minmax: [0, 0]
activity value minmax: [54.05453323119423, 395.67097167419934]
vid min? 88
NUM OF AVAILABLE TAXIS: 103
NUM OF REQUESTS: 86
testing [[  0  -1]
 [  0   0]
 [  0  37]
 ...
 [102  76]
 [102  79]
 

NEW_DEMAND_LIST, len: 0
Active time minmax: [0, 0]
activity value minmax: [54.05453323119423, 395.67097167419934]
vid min? 88
NUM OF AVAILABLE TAXIS: 103
NUM OF REQUESTS: 0
testing [[   0   -1]
 [   1   -2]
 [   2   -3]
 [   3   -4]
 [   4   -5]
 [   5   -6]
 [   6   -7]
 [   7   -8]
 [   8   -9]
 [   9  -10]
 [  10  -11]
 [  11  -12]
 [  12  -13]
 [  13  -14]
 [  14  -15]
 [  15  -16]
 [  16  -17]
 [  17  -18]
 [  18  -19]
 [  19  -20]
 [  20  -21]
 [  21  -22]
 [  22  -23]
 [  23  -24]
 [  24  -25]
 [  25  -26]
 [  26  -27]
 [  27  -28]
 [  28  -29]
 [  29  -30]
 [  30  -31]
 [  31  -32]
 [  32  -33]
 [  33  -34]
 [  34  -35]
 [  35  -36]
 [  36  -37]
 [  37  -38]
 [  38  -39]
 [  39  -40]
 [  40  -41]
 [  41  -42]
 [  42  -43]
 [  43  -44]
 [  44  -45]
 [  45  -46]
 [  46  -47]
 [  47  -48]
 [  48  -49]
 [  49  -50]
 [  50  -51]
 [  51  -52]
 [  52  -53]
 [  53  -54]
 [  54  -55]
 [  55  -56]
 [  56  -57]
 [  57  -58]
 [  58  -59]
 [  59  -60]
 [  60  -61]
 [  61  -62]
 [  62  -63]


...

DAY: 29
TRESH: 10
NUMREQ from original data: 74
ORIGINAL TRIP NODES: [2829, 7525, 792, 2196, 2830, 6580, 5475, 478, 976, 3521, 7525, 7561, 8014, 2612, 2162, 87, 9072, 2316, 2610, 2176, 258, 478, 3205, 7525, 2382, 7855, 9071, 5673, 9689, 3556, 5622, 8044, 2559, 5792, 259, 8228, 7525, 8044, 2421, 6741, 163, 7525, 6041, 2817, 1268, 4944, 478, 2822, 8077, 6745, 5620, 2440, 5673, 774, 3907, 6529, 7337, 8334, 87, 7525, 6798, 1700, 985, 3399, 2605, 4753, 8460, 7353, 7525, 8322, 3205, 7645, 5944, 1310]
how large the duplicates? (len vs lenset): (74, 62)
len v_init: 2119
HOW LARGE? 73
REQUIRED? 14
NUM OF TAXIS: 88
NUM OF REQS: 74
WEIGHT: 0
PERIOD 1
req_pos bf: 0
req_pos aft: 74
NEW_DEMAND_LIST, len: 74
Active time minmax: [0, 0]
activity value minmax: [57.23602763168342, 395.67097167419934]
vid min? 81
NUM OF AVAILABLE TAXIS: 88
NUM OF REQUESTS: 74
testing [[ 0 -1]
 [ 0  0]
 [ 0 40]
 ...
 [87 63]
 [87 65]
 [87 70]] [    0.          2330.97934045  2220.44466499  2508.362051
  5996.02745297     0

C:\Users\user\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


DAY: 30
TRESH: 10
NUMREQ from original data: 105
ORIGINAL TRIP NODES: [2732, 478, 9165, 2764, 2529, 7525, 3920, 5501, 1021, 2375, 8221, 1955, 9689, 1384, 4187, 5804, 6510, 2707, 3139, 81, 1196, 478, 6531, 8886, 5278, 1023, 5618, 1045, 1467, 976, 3381, 478, 6601, 6800, 1230, 3381, 7655, 7525, 4257, 1354, 1743, 5954, 2177, 6510, 4668, 1198, 2830, 6730, 8340, 163, 2375, 478, 478, 2179, 4263, 7700, 6601, 2707, 2531, 4318, 5229, 2531, 8955, 7525, 4388, 2529, 478, 7322, 8886, 2732, 7361, 1177, 2819, 2256, 7525, 1743, 7088, 1457, 7701, 3882, 9519, 7525, 2407, 5278, 7525, 6530, 8977, 7525, 7525, 6800, 91, 4188, 9062, 3508, 7370, 4251, 1715, 5962, 3688, 117, 1109, 8351, 8302, 2094, 9614]
how large the duplicates? (len vs lenset): (105, 81)
len v_init: 2281
HOW LARGE? 141
REQUIRED? 21
NUM OF TAXIS: 126
NUM OF REQS: 105
WEIGHT: 0
PERIOD 1
req_pos bf: 0
req_pos aft: 105
NEW_DEMAND_LIST, len: 105
Active time minmax: [0, 0]
activity value minmax: [51.527766009368705, 395.81002401455487]
vid min? 112

C:\Users\user\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


DAY: 31
TRESH: 10
NUMREQ from original data: 84
ORIGINAL TRIP NODES: [4423, 1955, 2717, 3563, 8725, 8337, 6156, 7525, 9231, 2415, 1310, 5812, 4393, 3205, 8621, 478, 2732, 5673, 2830, 7659, 1267, 7525, 7420, 1022, 1192, 2707, 1871, 3564, 3149, 1142, 478, 815, 2177, 7525, 4187, 5815, 3614, 6782, 1174, 2810, 7088, 1417, 2521, 2375, 6800, 7525, 5501, 3920, 6848, 3724, 6795, 2234, 8959, 3497, 2810, 5930, 2563, 2780, 2837, 8311, 2164, 3205, 1051, 1230, 7525, 4906, 5809, 1022, 3205, 2791, 5395, 817, 478, 1419, 7525, 6533, 2178, 9231, 7525, 5936, 478, 2605, 2055, 1266]
how large the duplicates? (len vs lenset): (84, 70)
len v_init: 2112
HOW LARGE? 35
REQUIRED? 16
NUM OF TAXIS: 100
NUM OF REQS: 84
WEIGHT: 0
PERIOD 1
req_pos bf: 0
req_pos aft: 84
NEW_DEMAND_LIST, len: 84
Active time minmax: [0, 0]
activity value minmax: [54.05453323119423, 395.67097167419934]
vid min? 88
NUM OF AVAILABLE TAXIS: 100
NUM OF REQUESTS: 84
testing [[ 0 -1]
 [ 0  0]
 [ 0 37]
 ...
 [99 70]
 [99 71]
 [99 76]] [   0.    

In [72]:
mask = np.array(All_edges)[:, 1] < 0
mask2 = np.array(Value_weight) == 0
mask3 = np.array(Distances) == 0
np.all(mask2 == mask3)

True

In [9]:
np.where(All_edges<0)

(array([   7,   13,   27,   38,   42,   51,   57,   66,   72,   81,   94,
         103,  107,  122,  124,  138,  146,  158,  171,  173,  179,  191,
         203,  214,  217,  229,  244,  250,  266,  274,  285,  293,  305,
         317,  326,  333,  343,  352,  364,  374,  380,  382,  390,  396,
         406,  414,  424,  436,  448,  463,  467,  472,  477,  493,  507,
         520,  529,  539,  550,  556,  558,  568,  581,  596,  607,  619,
         625,  640,  646,  661,  674,  690,  698,  715,  730,  740,  746,
         749,  757,  764,  770,  782,  786,  798,  811,  815,  827,  840,
         849,  861,  866,  881,  891,  906,  918,  932,  947,  953,  958,
         971,  982,  997, 1001, 1014, 1026, 1036, 1048, 1052, 1060, 1071,
        1083, 1095, 1108, 1123, 1137, 1151, 1164, 1178, 1194, 1212, 1229,
        1244, 1260, 1275, 1287, 1299, 1311, 1323, 1335, 1347, 1359],
       dtype=int64),
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1,